In [ ]:
from sympy import *
import numpy as np
from matplotlib import pyplot as plt
init_printing()

In [ ]:
# Differentialgleichungen höherer Ordnung

In [ ]:
x = Symbol('x')
y = Function('y')

In [ ]:
dgl = Eq(y(x).diff(x,2), -y(x).diff(x) - y(x))
dgl

In [ ]:
lsg = dsolve(dgl)
lsg

In [ ]:
mu = S('mu')
char_poly = dgl.subs(y(x), exp(mu*x)).doit()
char_poly

In [ ]:
solve(char_poly)

Anfangsbedingungen $y(1)=2$, $y'(1)=3$

In [ ]:
lsg = dsolve(dgl, ics={y(1): 2, y(x).diff(x).subs(x, 1): 3})
lsg

In [ ]:
phi = lsg.rhs
plot(phi);

# Besselfunktionen

In [ ]:
nu = S('nu')

In [ ]:
dgl = Eq(y(x).diff(x,2) + y(x).diff(x)/x + (1 - nu**2/x**2)*y(x), 0)
dgl

In [ ]:
lsg = dsolve(dgl)
lsg

In [ ]:
phi = lsg.rhs
print(phi)

$J_\nu$: Besselfunktion erster Art der Ordnung $\nu$

$Y_\nu$: Besselfunktion zweiter Art der Ordnung $\nu$, Webersche Funktion

In [ ]:
nu_n = 2
plot(besselj(nu_n, x), bessely(nu_n, x), (x, 0, 60), ylim=(-3,1));

Wronskische

In [ ]:
M = Matrix(2,2,[besselj(nu, x), bessely(nu,x), besselj(nu,x).diff(x), bessely(nu,x).diff(x)])
M

In [ ]:
M.det()

In [ ]:
W = M.det().simplify()
W

In [ ]:
plot(W.subs(nu, 3), (x, .2, 10));

Welche Funktion das ist, sehen wir mit der Reihenentwicklung

Allerdings müssen wir diese Entwicklung selber implementieren

https://dlmf.nist.gov/10.2#E2

In [ ]:
k = S('k')

In [ ]:
a = (-1)**k * (x/2)**(2*k) / factorial(k) / gamma(nu+1+k)

In [ ]:
def Bj_ser(nu, x, N):
    k = S('k')
    a = (-1)**k * (x/2)**(2*k) / factorial(k) / gamma(nu+1+k)
    res = 0
    for kk in range(N):
        res += a.subs(k, kk)
    return res * (x/2)**nu

In [ ]:
Bj_ser(nu, x, 10)

In [ ]:
Bj_ser(-5.5, 2, 10)

In [ ]:
besselj(-5.5, 2).n()

In [ ]:
W

In [ ]:
connection_formula =  {bessely(nu,x): (besselj(nu,x)*cos(nu*pi)-besselj(-nu,x))/sin(nu*pi)}
connection_formula

In [ ]:
ers = {nu: nu+1}
for key in list(connection_formula):
    connection_formula[key.subs(ers)] = connection_formula[key].subs(ers)
connection_formula

In [ ]:
W1 = W.subs(connection_formula)
W1

In [ ]:
ers = {}
for mu in [nu, -nu, nu+1, -nu-1]:
    ers[besselj(mu, x)] = Bj_ser(mu, x, 8)
ers

In [ ]:
W2 = W1.subs(ers)
W2

In [ ]:
W3 = W2.expand(trig=True) + O(x**8)
W3

In [ ]:
simplify(W3.removeO())

# Gekoppelte Pendel

In [ ]:
y = Function('y')
w = Function('w')
t = Symbol('t', positive=True)

In [ ]:
a = Rational(1,7)
b = Rational(1,6)

In [ ]:
dgl1 = Eq(y(t).diff(t,2), a*w(t) - (1+a)*y(t))
dgl2 = Eq(w(t).diff(t,2), b*y(t) - (1+b)*w(t))
dgs = {dgl1, dgl2}
dgs

In [ ]:
lsg = dsolve(dgs)
lsg

In [ ]:
phi_c = lsg[0].rhs
psi_c = lsg[1].rhs

Bestimme reelles Fundamentalsystem

In [ ]:
C = symbols('C1, C2, C3, C4')
ersetzungen = []
for j in range(4):
    ers = {C[j]: 1}
    for i in range(4):
        if j != i:
            ers[C[i]] = 0
    ersetzungen.append(ers)

In [ ]:
ersetzungen

In [ ]:
fus = []
for s in ersetzungen:
    fus.append([phi_c.subs(s), 
                psi_c.subs(s)])

In [ ]:
fus

In [ ]:
f = fus[0][0] + fus[2][0]
g = fus[0][1] + fus[2][1]
f, g

In [ ]:
plot(f,g+3, (t,-100,100));